<a href="https://colab.research.google.com/github/OwaisBmsce/AI_LAB/blob/main/BIS_SEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from scipy.special import gamma # Import gamma function from scipy.special

def levy_flight(dim):
    """Generates a step based on Lévy flight distribution"""
    beta = 1.5  # Parameter for Lévy distribution
    sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) /
             (gamma((1 + beta) / 2) * beta * (2 ** ((beta - 1) / 2)))) ** (1 / beta)

    u = np.random.randn(dim) * sigma
    v = np.random.randn(dim)

    step = u / (np.abs(v) ** (1 / beta))
    return step

def cuckoo_search(objective_func, bounds, n_nests=20, pa=0.25, max_iter=100):
    """Implementation of the Cuckoo Search algorithm.

    Returns:
        A tuple (best_solution, best_fitness).
    """
    dim = len(bounds)

    # Initialize nests randomly within bounds
    nests = np.array([[np.random.uniform(low=bounds[d][0], high=bounds[d][1]) for d in range(dim)]
                      for _ in range(n_nests)])

    # Evaluate initial nests
    fitness = np.array([objective_func(nest) for nest in nests])

    best_nest_idx = np.argmin(fitness)
    best_solution = nests[best_nest_idx]
    best_fitness = fitness[best_nest_idx]

    print(f"Initial Best Fitness: {best_fitness}")

    for iteration in range(max_iter):
        # 1. Generate new cuckoo solutions via Lévy flights
        new_nests = np.copy(nests)
        for i in range(n_nests):
            # Select a cuckoo (existing nest) to generate a new one from
            cuckoo_nest = nests[i]

            # Generate new solution using Lévy flight
            step_size = levy_flight(dim)
            # Scale step size (can be tuned)
            alpha = 0.1
            new_solution = cuckoo_nest + alpha * step_size * (cuckoo_nest - best_solution) # Use best_solution to guide

            # Apply bounds to new solution
            for d in range(dim):
                new_solution[d] = np.clip(new_solution[d], bounds[d][0], bounds[d][1])

            new_fitness = objective_func(new_solution)

            # Randomly select a host nest to compare with
            j = np.random.randint(0, n_nests)

            # If new cuckoo solution is better, replace the host nest
            if new_fitness < fitness[j]:
                new_nests[j] = new_solution
                fitness[j] = new_fitness

        nests = new_nests # Update nests after cuckoo generation

        # Update global best
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_fitness:
            best_solution = nests[current_best_idx]
            best_fitness = fitness[current_best_idx]

        # 2. Abandon a fraction of worst nests and build new ones
        n_abandon = int(np.ceil(pa * n_nests))
        if n_abandon > 0:
            # Sort nests by fitness (ascending) and get indices of worst nests
            worst_nests_indices = np.argsort(fitness)[::-1][:n_abandon]

            for idx in worst_nests_indices:
                # Generate a new nest randomly within bounds
                new_random_nest = np.array([np.random.uniform(low=bounds[d][0], high=bounds[d][1]) for d in range(dim)])
                nests[idx] = new_random_nest
                fitness[idx] = objective_func(new_random_nest)

        # Update global best again after abandonment
        current_best_idx = np.argmin(fitness)
        if fitness[current_best_idx] < best_fitness:
            best_solution = nests[current_best_idx]
            best_fitness = fitness[current_best_idx]

        if (iteration + 1) % 10 == 0:
            print(f"Iteration {iteration+1}/{max_iter}, Best Fitness: {best_fitness}")

    return best_solution, best_fitness


# --- Example Usage: Minimizing a simple function (e.g., Sphere function) ---
def sphere_function(x):
    return np.sum(x**2)

# Define the bounds for each dimension (e.g., -5 to 5 for 2 dimensions)
bounds = [(-5, 5), (-5, 5)]

# Run Cuckoo Search
best_sol, min_val = cuckoo_search(sphere_function, bounds, n_nests=25, pa=0.2, max_iter=50)

print(f"\nBest solution found: {best_sol}")
print(f"Minimum value found: {min_val}")


Initial Best Fitness: 2.3630683621820947
Iteration 10/50, Best Fitness: 0.007807603448506275
Iteration 20/50, Best Fitness: 0.007807603448506275
Iteration 30/50, Best Fitness: 0.007807603448506275
Iteration 40/50, Best Fitness: 0.007807603448506275
Iteration 50/50, Best Fitness: 0.007807603448506275

Best solution found: [-0.07071796 -0.0529771 ]
Minimum value found: 0.007807603448506275
